一、訓練資料清洗

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data=pd.read_csv("train.csv")
train_data

,ID,Date,Temp_m,Generation,Irradiance,Capacity,Lat,Lon,Angle,Irradiance_m,Temp,Module
0,1,2020/6/9,NaN,630,8.37,499.80,25.11,121.26,22.00,2325,26.3,1
1,2,2020/6/10,NaN,2246,17.11,499.80,25.11,121.26,22.00,4753,27.9,1
2,3,2020/6/11,NaN,1147,12.26,499.80,25.11,121.26,22.00,3406,28.0,1
3,4,2020/6/12,NaN,2946,21.51,499.80,25.11,121.26,22.00,5975,28.6,1
4,5,2020/6/13,NaN,3139,23.18,499.80,25.11,121.26,22.00,6439,29.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3579,3580,2021/10/28,40.9,2285,18.73,498.56,24.04,120.52,2.21,131913,25.0,2
3580,3581,2021/10/28,43.7,423,18.73,99.20,24.08,120.50,1.76,459967,25.0,2
3581,3582,2021/10/28,44.4,1496,18.73,343.20,24.08,120.52,-2.62,107142,25.0,3
3582,3583,2021/10/28,47.5,450,18.73,99.84,24.07,120.48,10.35,431817,25.0,2


In [3]:
train_data.dtypes

ID                int64
Date             object
Temp_m          float64
Generation        int64
Irradiance      float64
Capacity        float64
Lat             float64
Lon             float64
Angle           float64
Irradiance_m      int64
Temp            float64
Module            int64
dtype: object

In [4]:
train_data=train_data.drop(columns=["Date"],axis=1)
train_data

,ID,Temp_m,Generation,Irradiance,Capacity,Lat,Lon,Angle,Irradiance_m,Temp,Module
0,1,NaN,630,8.37,499.80,25.11,121.26,22.00,2325,26.3,1
1,2,NaN,2246,17.11,499.80,25.11,121.26,22.00,4753,27.9,1
2,3,NaN,1147,12.26,499.80,25.11,121.26,22.00,3406,28.0,1
3,4,NaN,2946,21.51,499.80,25.11,121.26,22.00,5975,28.6,1
4,5,NaN,3139,23.18,499.80,25.11,121.26,22.00,6439,29.1,1
...,...,...,...,...,...,...,...,...,...,...,...
3579,3580,40.9,2285,18.73,498.56,24.04,120.52,2.21,131913,25.0,2
3580,3581,43.7,423,18.73,99.20,24.08,120.50,1.76,459967,25.0,2
3581,3582,44.4,1496,18.73,343.20,24.08,120.52,-2.62,107142,25.0,3
3582,3583,47.5,450,18.73,99.84,24.07,120.48,10.35,431817,25.0,2


In [5]:
train_data.isna().sum()

ID                 0
Temp_m          1458
Generation         0
Irradiance        24
Capacity           0
Lat                0
Lon                0
Angle              0
Irradiance_m       0
Temp              15
Module             0
dtype: int64

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [7]:
train_copy=train_data.copy()

In [8]:
sindex=np.argsort(train_copy.isna().sum().values.tolist())

In [9]:
for i in sindex:
    if train_copy.iloc[:,i].isna().sum()==0:
        continue
    df1=train_copy
    fillc=df1.iloc[:,i]
    df2=df1.iloc[:,df1.columns !=df1.columns[i]]
    df3=SimpleImputer(missing_values=np.nan,strategy="constant",fill_value=0).fit_transform(df2)

    Ytrain = fillc[fillc.notnull()]
    Ytest = fillc[fillc.isnull()] 
    Xtrain = df3[Ytrain.index,:]
    Xtest = df3[Ytest.index,:]

    rfc=RandomForestRegressor().fit(Xtrain,Ytrain)
    Y_pred=rfc.predict(Xtest)

    train_copy.loc[train_copy.iloc[:,i].isnull(),train_copy.columns[i]]=Y_pred

In [10]:
train_copy.isnull().sum()

ID              0
Temp_m          0
Generation      0
Irradiance      0
Capacity        0
Lat             0
Lon             0
Angle           0
Irradiance_m    0
Temp            0
Module          0
dtype: int64

In [11]:
train_copy.to_csv("Final_Train.csv",index=False,encoding="utf-8-sig")

二、測試資料清洗

In [12]:
test_data=pd.read_csv("test.csv")
test_data

,ID,Date,Temp_m,Generation,Irradiance,Capacity,Lat,Lon,Angle,Irradiance_m,Temp,Module
0,1,2021/10/29,NaN,NaN,18.32,283.2,24.980,121.03,-31.00,5089,22.0,4
1,2,2021/10/29,NaN,NaN,8.72,499.8,25.110,121.26,22.00,2422,22.2,1
2,3,2021/10/29,NaN,NaN,8.72,438.3,25.110,121.26,-160.00,2422,22.2,1
3,4,2021/10/29,NaN,NaN,18.32,498.6,25.030,121.08,-95.00,5089,22.6,1
4,5,2021/10/29,38.7,NaN,17.10,492.8,24.107,120.44,4.63,121729,23.9,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1534,1535,2022/2/16,35.3,NaN,20.16,343.2,24.080,120.52,-2.62,126196,17.2,3
1535,1536,2022/2/17,NaN,NaN,6.47,499.8,25.110,121.26,22.00,1797,15.8,1
1536,1537,2022/2/17,NaN,NaN,6.47,438.3,25.110,121.26,-160.00,1797,15.8,1
1537,1538,2022/2/17,NaN,NaN,6.98,283.2,24.980,121.03,-31.00,1939,15.9,4


In [13]:
test_data.dtypes

ID                int64
Date             object
Temp_m          float64
Generation      float64
Irradiance      float64
Capacity        float64
Lat             float64
Lon             float64
Angle           float64
Irradiance_m      int64
Temp            float64
Module            int64
dtype: object

In [14]:
test_data=test_data.drop(columns=["Date","Generation"],axis=1)
test_data

,ID,Temp_m,Irradiance,Capacity,Lat,Lon,Angle,Irradiance_m,Temp,Module
0,1,NaN,18.32,283.2,24.980,121.03,-31.00,5089,22.0,4
1,2,NaN,8.72,499.8,25.110,121.26,22.00,2422,22.2,1
2,3,NaN,8.72,438.3,25.110,121.26,-160.00,2422,22.2,1
3,4,NaN,18.32,498.6,25.030,121.08,-95.00,5089,22.6,1
4,5,38.7,17.10,492.8,24.107,120.44,4.63,121729,23.9,2
...,...,...,...,...,...,...,...,...,...,...
1534,1535,35.3,20.16,343.2,24.080,120.52,-2.62,126196,17.2,3
1535,1536,NaN,6.47,499.8,25.110,121.26,22.00,1797,15.8,1
1536,1537,NaN,6.47,438.3,25.110,121.26,-160.00,1797,15.8,1
1537,1538,NaN,6.98,283.2,24.980,121.03,-31.00,1939,15.9,4


In [15]:
test_data.isna().sum()

ID                0
Temp_m          440
Irradiance        0
Capacity          0
Lat               0
Lon               0
Angle             0
Irradiance_m      0
Temp            183
Module            0
dtype: int64

In [16]:
test_copy=test_data.copy()

In [17]:
kindex=np.argsort(test_copy.isna().sum().values.tolist())

In [18]:
for i in kindex:
    if test_copy.iloc[:,i].isna().sum()==0:
        continue
    df4=test_copy
    fillc=df4.iloc[:,i]
    df5=df4.iloc[:,df4.columns !=df4.columns[i]]
    df6=SimpleImputer(missing_values=np.nan,strategy="constant",fill_value=0).fit_transform(df5)

    Ytrain = fillc[fillc.notnull()]
    Ytest = fillc[fillc.isnull()] 
    Xtrain = df6[Ytrain.index,:]
    Xtest = df6[Ytest.index,:]

    rfc=RandomForestRegressor().fit(Xtrain,Ytrain)
    Y_pred=rfc.predict(Xtest)

    test_copy.loc[test_copy.iloc[:,i].isnull(),test_copy.columns[i]]=Y_pred

In [19]:
test_copy.isnull().sum()

ID              0
Temp_m          0
Irradiance      0
Capacity        0
Lat             0
Lon             0
Angle           0
Irradiance_m    0
Temp            0
Module          0
dtype: int64

In [20]:
test_copy.to_csv("Final_Test.csv",index=False,encoding="utf-8-sig")